In [1]:
from bert_score import score
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import time

from overlap_evaluate import read_references, read_annotations, ANNOTATIONS, _print_score

from evaluate import *


/Users/calebkumar/repos/stanford/project/odqa-experiments/odqa-experiments/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "google/t5-xl-ssm-nq"
# model_name = "google/t5-large-ssm-nq"

# t5_qa_model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-large-ssm-nq")
# t5_tok = AutoTokenizer.from_pretrained("google/t5-large-ssm-nq")

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
class SampleQA(QAModel):
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def predict_answer(self, question: str, contexts: list) -> str:
        input_ids = self.tokenizer(question, return_tensors="pt").input_ids
        gen_output = self.model.generate(input_ids)[0]
        prediction = self.tokenizer.decode(gen_output, skip_special_tokens=True)

        return prediction
        
        

In [5]:
model = SampleQA(dataset=QADataset.nq)

In [6]:
predictions = model.get_predictions()

Retrieving 3610 predictions
At example: 40 after 108.706 seconds. Expecting to take: 161.7002 more minutes
At example: 80 after 212.465 seconds. Expecting to take: 156.2503 more minutes
At example: 120 after 319.34 seconds. Expecting to take: 154.7912 more minutes
At example: 160 after 427.823 seconds. Expecting to take: 153.7489 more minutes
At example: 200 after 527.089 seconds. Expecting to take: 149.7811 more minutes
At example: 240 after 634.91 seconds. Expecting to take: 148.5866 more minutes
At example: 280 after 780.187 seconds. Expecting to take: 154.6442 more minutes
At example: 320 after 1026.825 seconds. Expecting to take: 175.9507 more minutes
At example: 360 after 1247.56 seconds. Expecting to take: 187.7116 more minutes
At example: 400 after 1469.877 seconds. Expecting to take: 196.596 more minutes
At example: 440 after 1723.213 seconds. Expecting to take: 206.9161 more minutes
At example: 480 after 1975.257 seconds. Expecting to take: 214.672 more minutes
At example: 52

In [ ]:
saved_results, label_scores = model.evaluate(get_bert_score=True, get_predictions=False)

Obtaining scores:

Scoring annotation label: total


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
# saved_results[0:5]

In [ ]:
df = pd.DataFrame(saved_results)

df.to_csv(f"results/nq/t5-xl-ssm-nq_results.csv", index=False)

In [ ]:
import json

with open("results/nq/t5-xl-ssm-nq_results.json", "w") as outfile:
    json.dump(label_scores, outfile)

In [5]:
# test = pd.read_csv(f"results/nq/t5-large-ssm-nq_results.csv").to_dict('records')

In [6]:
# from evaluate import *

In [9]:
# model.compute_no_overlap_score(test)

{'em': 0.0224,
 'f1': 0.075,
 'bert_score': 0.2455,
 'meteor_score': 0.0573,
 'total': 357}

In [10]:
# model.compute_answer_overlap_only_scores(test)

{'em': 0.1333,
 'f1': 0.1977,
 'bert_score': 0.4635,
 'meteor_score': 0.1645,
 'total': 315}

In [ ]:
# from transformers import BigBirdForQuestionAnswering

# model = BigBirdForQuestionAnswering.from_pretrained("google/bigbird-base-trivia-itc")

# question = "How many days are in a year?"
# context = "There are 365 days in a year."
# encoded_input = tokenizer(question, context, return_tensors='pt')
# output = model(**encoded_input)

# tokenizer.decode(output)